# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
# импорт библиотеки pandas
import pandas as pd

# импорт контейнера SnowballStemmer из модуля nltk.stem
from nltk.stem import SnowballStemmer

# Передаём стеммеру как аргумент значение 'russian', чтобы включить лингвистический анализ именно русского языка
russian_stemmer = SnowballStemmer('russian')

# импорт контейнера Mystem из модуля pymystem3
from pymystem3 import Mystem
m = Mystem()

# импорт контейнера Counter из модуля collections
from collections import Counter

# чтение файла с данными и сохранение в data
data = pd.read_csv('/datasets/data.csv')

# получение первых 10 строк таблицы data
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
# получение первых 10 строк таблицы data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Итак, в таблице 12 столбцов. Типы данных во всех столбцах — `float64`, `int64`, `object`.

Согласно документации к данным:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В названиях колонок не видны нарушения стиля:

В данных видны проблемы:
1. Количество значений в столбцах различается. В данных `days_employed` и `total_income` - есть пропущенные значения.
2. Колонка `days_employed`(стаж в днях) имеет тип данных - `float64`, трудовой стаж должен быть в целых днях. Значит в данной колонке есть ошибки.
3. В колонке `education` - неявные дубликаты названия образования заемщика. Это могут быть названия образований с ошибками, символы в разном регистре или альтернативные названия того же жанра.
4. В колонке `purpose` - неявные дубликаты.

In [ ]:
# поиск артефактов в столбце children
data['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20])

Артефакты найдены: "-1" и "20" детей заёмщик иметь не может.

In [ ]:
# поиск артефактов в столбце dob_years
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

Артефакт найден. Возраст заёмщика не может быть нулем.

In [ ]:
# поиск артефактов в столбце education
data['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [ ]:
for col in data.select_dtypes(include='object').columns:
    print('Уникальные значения столбца', col)
    print(data[col].sort_values().unique(), '\n')

Уникальные значения столбца education
['высшее' 'начальное' 'неоконченное высшее' 'среднее' 'ученая степень'] 

Уникальные значения столбца family_status
['в разводе' 'вдовец / вдова' 'гражданский брак' 'женат / замужем'
 'не женат / не замужем'] 

Уникальные значения столбца gender
['F' 'M' 'XNA'] 

Уникальные значения столбца income_type
['безработный' 'в декрете' 'госслужащий' 'компаньон' 'пенсионер'
 'предприниматель' 'сотрудник' 'студент'] 

Уникальные значения столбца purpose
['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупка жилья для семьи'
 'п

Артефактов нет. Только дупликаты.

In [ ]:
# поиск артефактов в столбце education_id
data['education_id'].sort_values().unique()

array([0, 1, 2, 3, 4])

Артефактов нет.

In [ ]:
# поиск артефактов в столбце family_status
data['family_status'].sort_values().unique()

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

Артефактов нет. Но есть проблема стиля записи 'Не женат / не замужем'.

In [ ]:
# поиск артефактов в столбце family_status_id
data['family_status_id'].sort_values().unique()

array([0, 1, 2, 3, 4])

Артефактов нет.

In [ ]:
# поиск артефактов в столбце gender
data['gender'].sort_values().unique()

array(['F', 'M', 'XNA'], dtype=object)

Артефактов найден. XNA указывает на неуказанный пол. В нашем исследовании данные по гендеру заемщика не понадобятся.

In [ ]:
# поиск артефактов в столбце income_type
data['income_type'].sort_values().unique()

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

Артефактов нет.

In [ ]:
# поиск артефактов в столбце income_type
data['debt'].sort_values().unique()

array([0, 1])

Артефактов нет.

In [ ]:
# поиск артефактов в столбце purpose
data['purpose'].sort_values().unique()

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

Артефактов нет. Только дупликаты.

**Вывод**

В каждой строке таблицы — данные о заемщике.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в количественных переменных. В колонке трудового стажа - ошибки и дубликаты в колонках: `education`, `purpose`.


Артефакты "-1" и "20" в колонке - `children`. Возможно отрицательное значение детей "-1" это ошибочно введенное "1", а "20" ошибочно преобразованное из "2.0".
Артефакт "0" в колонке `dob_years`. Возраст заёмщика не может быть нулем.
Артефакт "XNA" в колонке `gender` указывает на неуказанный пол.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
# подсчёт пропусков
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах `days_employed` и `total_income` у нас пропуски. Пропуски в таких переменных заполняют характерными значениями.
Возможно причина пропуска значений в `days_employed` ошибка при преобразовании данных.

In [ ]:
# посмотрим на столбец days_employed
data['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

Значения дней стажа заёмщика выглядят нереалистично. У нас присутствуют отрицательные значения, огромные значения и значения дней стажа с дробью.

Проверим цели исследования:
* Есть ли зависимость между наличием детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

В нашем исследовании зависимости от стажа заёмщика нет. Не будем тратить время на детальную обработку пропусков столбца `days_employed`, преобразуем отрицательные значения на положительные и заменим пропущеные значения на среднее значение столбца.

In [ ]:
# преобразуем отрицательные значения в положительные в days_employed
data['days_employed'] = abs(data['days_employed'])

# поиск среднего значения в days_employed
temp_days_employed = data['days_employed'].mean()

# замена пропущенных значений на среднее
data['days_employed'] = data['days_employed'].fillna(temp_days_employed)

Убрали пропуски в столбце `days_employed` заменив отрицательные значения на абсолютные и найдня среднее число.

In [ ]:
# посмотрим на минимальное и максимальное значения в столбеце total_income
print (f"минимальное значение: {data['total_income'].min()} максимальное значение: {data['total_income'].max()}")

минимальное значение: 20667.26379327158 максимальное значение: 2265604.028722744


В нашем исследовании учитывается уровень дохода заёмщика. Для обработки пропусков в столбце `total_income` посмотрим на значения в нём. Найдя минимальное и максимальное мы можем сделать вывод, что данные в столбце реалистичные.

In [ ]:
# поиск медианого значения в total_income
temp_total_income = data['total_income'].median()

# замена пропущенных значений на медианое
data['total_income'] = data['total_income'].fillna(temp_total_income)

Убрали пропуски в столбце `total_income` заменив на медианые.

In [ ]:
# убеждаемся что пропусков не осталось
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [ ]:
# поиск артефактов в столбце children
data['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20])

Проверяем артефакты в столбце `children`.

In [ ]:
# изменяем артефактов в столбце children
data['children'] = abs(data['children'])

# проверяем результат
data['children'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5, 20])

Отрицательное количество детей у заемщика скорее всего ошибка при вводе данных, сделали число детей из отрицательных - положительными.

In [ ]:
# поиск артефактов в столбце family_status
data['family_status'].sort_values().unique()

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

Проверяем артефакты в столбце `family_status`.

In [ ]:
# приведем названия в family_status к единому стилю и одному регистру символов
data['family_status'] = data['family_status'].str.lower()
data['family_status'].sort_values().unique()

array(['в разводе', 'вдовец / вдова', 'гражданский брак',
       'женат / замужем', 'не женат / не замужем'], dtype=object)

Артефактов нет. Но приведем категории семейного статуса к единому стилю в нижнем регистре.

**Вывод**

Не все пропущенные значения влияют на исследование. Так в `days_employed` пропуски не важны для данного проекта. Достаточно заменить их на абсолютные средние значения.

Но пропуски в `total_income` могут помешать поиску зависимости между уровнем дохода и возвратом кредита в срок. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Придётся:
* заполнить эти пропуски медиаными обозначениями,
* оценить, насколько они повредят расчётам.

### Замена типа данных

In [ ]:
# меняем тип данных в колонках общего трудового стажа в днях и ежемесячного дохода на целочисленный
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

**Вывод**

Мы выбрали метод astype для изменения типа данных на целые числа т.к. данные в столбцах однородные вещественные числа не содержащие некорректные значения.

### Обработка дубликатов

Из первого шага с изучением общей информации мы нашли неявные дубликаты в столбце уровеня образования клиента `education` которые отличаются регистром.

In [ ]:
# для наглядности отображаем все значения уровня образования заемщика
data['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

In [ ]:
# изменяем регистр записи символов в уровнях образования клиента
data['education'] = data['education'].str.lower()

# проверяем результат
data['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

In [ ]:
# проверяем количество полных дубликатов
print(data.duplicated().sum())

# удаление явных дубликатов (с удалением старых индексов и формированием новых)
data = data.drop_duplicates().reset_index(drop=True)

# проверяем результат
data.duplicated().sum()

71


0

Ищем полные дубликаты стандартным методом из первого курса и удаляем их.

**Вывод**

Для поиска проанализировали уникальные значения в столбцах, для удаления неявных дубликатов использовали преобразование символов к единому регистру.
Причиной возникшим неявным дубликатам являются написания уровня образования заемщика в разных регистрах символов.

Далее были найдены полные дубликаты, причиной могло стать повторное заполнение заявок клиентами.

### Лемматизация

In [ ]:
# смотрим, насколько часто отдельная лемма встречается во всем датафрейме
lemmas = m.lemmatize(' '.join(data['purpose']))
Counter(lemmas)

Counter({'покупка': 5897,
         ' ': 55023,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

Наиболее встречающие леммы: `автомобиль`, `образование`, `свадьба`, `ремонт`, `недвижимость`, `жилье`.

In [ ]:
# выделяем наиболее часто встречающиеся леммы в список lemmas_purposes
lemmas_purposes = ['жилье','недвижимость','строительство',"автомобиль",'образование',"свадьба","ремонт"]

# создаем функцию, которая будет возвращать категорию, в зависимости от того, что содержится в строке
"""
в функции пробегаем по лемматизируемому значению из столбца с целями кредита purpose
если находим совпадение с значением в нашем списке lemmas_purposes то этот тип кредита выбираем
"""

def categoryzer_purpose(value):
    try:
        for lemma_purpose in m.lemmatize(value):
            if lemma_purpose in lemmas_purposes:
                return lemma_purpose
    except:
        return 'функция categoryzer_purpose не сработала'

# создаем новый столбец purpose_type в датафрейме, с категориями целей кредита и применяем к ней нашу функцию categoryzer_purpose
data['purpose_type'] = data['purpose'].apply(categoryzer_purpose)

# поиск всех уникальных значений в столбце purpose_type для проверки, что функция categoryzer_purpose отработала
data['purpose_type'].sort_values().unique()

array(['автомобиль', 'жилье', 'недвижимость', 'образование', 'ремонт',
       'свадьба', 'строительство'], dtype=object)

1. Выделили наиболее часто встречающиеся леммы в список.
2. Создали функцию, которая будет возвращать категорию, в зависимости от того, что содержится в строке.
3. Создали новый столбец, с категориями целей кредита.
4. Проверили значения в новом столбце.

In [ ]:
# убираем неявные дубликаты названия категорий 
data['purpose_type'].replace("жилье",'недвижимость',inplace = True)
data['purpose_type'].replace("ремонт",'недвижимость',inplace = True)
data['purpose_type'].replace("строительство",'недвижимость',inplace = True)

# поиск всех уникальных значений в столбце purpose_type для проверки, что неявные дубликаты в названиях убраны
data['purpose_type'].sort_values().unique()

array(['автомобиль', 'недвижимость', 'образование', 'свадьба'],
      dtype=object)

Типы причин на кредит: `жилье`, `ремонт`, `строительство` относятся к одной категории `недвижимость`. 

Убрали лишние причины и убедились в корректности данных проверив столбец с типами причин на кредиты.

**Вывод**

Мы лемматизировали цели получения кредитов, чтобы категоризовать их по типу. Мы создали отдельный столбец с типами целей получения кредитов. Мы получили 4 категории целей кредита.

### Категоризация данных

Для категоризации данных взглянем на гипотезы исследования:

* Есть ли зависимость между наличием детей и возвратом кредита в срок?
`Нужна категория заемщиков - с наличием детей и без наличия.`

* Есть ли зависимость между семейным положением и возвратом кредита в срок?
`Дополнительных категорий - не понадобится.`

* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
`Нужна категория заемщиков - с разным уровнем дохода.`

* Как разные цели кредита влияют на его возврат в срок?
`Дополнительных категорий - не понадобится.`

In [ ]:
# выделяем категорию заемщиков с детьми и без детей

# напишем функцию для категоризации заемщиков по детям
def having_children(value):
    try:
        if value == 0:
            return 'нет'
        else:
            return 'да'
    except:
        return 'ошибка в типе данных'
        
# добавим в датафрейм новый столбец со статусом наличия детей у заемщика
data['children_status'] = data['children'].apply(having_children)

Добавили столбец со статусом наличия детей у заёмщика

In [ ]:
# посмотрим на значения в total_income

print('количество значений:', data['total_income'].count())
print('минимальное значение:', data['total_income'].min())
print('максимальное значение:', data['total_income'].max())
print('медианое значение:', int(data['total_income'].median()))
print('среднее значение:', int(data['total_income'].mean()))

количество значений: 21454
минимальное значение: 20667
максимальное значение: 2265604
медианое значение: 145017
среднее значение: 165225


Для категоризации заемщиков по уровню дохода посмотрим на значения в total_income.
Заемщиков без дохода - нет.
Медианое значение `145017`, а в среднем наши заемщики получают `165523`. Используем данные числа для разделения заемщиков на уровни дохода.

In [ ]:
# выделяем категорию заемщиков с низким, средним и высоким доходом

# напишем функцию для категоризации заемщиков
def total_income_group(value):
    try:
        if value < 145017:
            return 'низкий'
        if value < 165523:
            return 'средний'
        return 'высокий'
    except:
        return 'ошибка в типе данных'

# добавим в датафрейм новый столбец со статусом доходп заемщика
data['total_income_group_status'] = data['total_income'].apply(total_income_group)

**Вывод**

Мы категоризировали данные, чтобы проверить наши гипотезы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
# для анализа создадим новую таблицу и проведем нужные нам расчеты

# создаем пустую таблицу debt_of_children_status
debt_of_children_status = pd.DataFrame()

# добавляем столбец суммы ячеек долга сгруппированных по наличию детей из основной таблицы data
debt_of_children_status['sum_children_status'] = data.groupby('children_status')['debt'].sum()

# добавляем столбец количества ячеек долга сгруппированных по наличию детей из основной таблицы data
debt_of_children_status['count_children_status'] = data.groupby('children_status')['debt'].count()

# вычесляем отношение количества к сумме и умножаем на 100
debt_of_children_status['result_children_status'] = debt_of_children_status['sum_children_status'] / debt_of_children_status['count_children_status'] * 100

# отображаем результат для анализа
debt_of_children_status.sort_values('result_children_status', ascending = False)['result_children_status']

children_status
да     9.208203
нет    7.543822
Name: result_children_status, dtype: float64

In [ ]:
df_example = data.groupby('children')['debt'].agg(['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children,,,,
0,14091,1063,7.54%,92.46%
1,4855,445,9.17%,90.83%
2,2052,194,9.45%,90.55%
3,330,27,8.18%,91.82%
4,41,4,9.76%,90.24%
5,9,0,0.00%,100.00%
20,76,8,10.53%,89.47%


Мы видим, что заемщики без детей чаще выплачивают кредит в срок, чем заемщики с детьми.

**Вывод**

Отвечаем на гипотезу: `Есть ли зависимость между наличием детей и возвратом кредита в срок?`

Да, зависимость между наличием детей и возвратом кредита в срок - есть.

Мы видим, что заемщики без детей чаще выплачивают кредит в срок, чем заемщики с детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
# для анализа создадим новую таблицу и проведем нужные нам расчеты

# создаем пустую таблицу debt_of_family_status
debt_of_family_status = pd.DataFrame()

# добавляем столбец суммы ячеек долга сгруппированных по семейным статусам из основной таблицы data
debt_of_family_status['sum_family_status'] = data.groupby('family_status')['debt'].sum()

# добавляем столбец количества ячеек долга сгруппированных по семейным статусам из основной таблицы data
debt_of_family_status['count_family_status'] = data.groupby('family_status')['debt'].count()

# вычесляем отношение количества к сумме и умножаем на 100
debt_of_family_status['result_family_status'] = debt_of_family_status['sum_family_status'] / debt_of_family_status['count_family_status'] * 100

# отображаем результат для анализа
debt_of_family_status.sort_values('result_family_status', ascending = False)['result_family_status']

family_status
не женат / не замужем    9.750890
гражданский брак         9.347145
женат / замужем          7.545182
в разводе                7.112971
вдовец / вдова           6.569343
Name: result_family_status, dtype: float64

**Вывод**

Отвечаем на гипотезу: `Есть ли зависимость между семейным положением и возвратом кредита в срок?`

Да, зависимость между семейным положением и возвратом кредита в срок - есть.

Не женатые заемщики и в гражданском браке имеют больший процент невозвратов кредитов в срок по сравнению с теми кто в разводе или овдовели.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
# для анализа создадим новую таблицу и проведем нужные нам расчеты

# создаем пустую таблицу debt_of_total_income_group_status
debt_of_total_income_group_status = pd.DataFrame()

# добавляем столбец суммы ячеек долга сгруппированных по уровню дохода из основной таблицы data
debt_of_total_income_group_status['sum_total_income_group_status'] = data.groupby('total_income_group_status')['debt'].sum()

# добавляем столбец количества ячеек долга сгруппированных по уровню дохода из основной таблицы data
debt_of_total_income_group_status['count_total_income_group_status'] = data.groupby('total_income_group_status')['debt'].count()

# вычесляем отношение количества к сумме и умножаем на 100
debt_of_total_income_group_status['result_total_income_group_status'] = debt_of_total_income_group_status['sum_total_income_group_status'] / debt_of_total_income_group_status['count_total_income_group_status'] * 100

# отображаем результат для анализа
debt_of_total_income_group_status.sort_values('result_total_income_group_status', ascending = False)['result_total_income_group_status']

total_income_group_status
средний    8.413926
низкий     8.310078
высокий    7.706398
Name: result_total_income_group_status, dtype: float64

**Вывод**

Отвечаем на гипотезу: `Есть ли зависимость между уровнем дохода и возвратом кредита в срок?`

Да, зависимость между уровнем дохода и возвратом кредита в срок - есть.

Мы видим, что заемщики с средним доходом чаще не отдают кредит в срок, чем заемщики с низким и высоким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
# для анализа создадим новую таблицу и проведем нужные нам расчеты

# создаем пустую таблицу debt_of_purpose
debt_of_purpose = pd.DataFrame()

# добавляем столбец суммы ячеек долга сгруппированных по целям кредита из основной таблицы data
debt_of_purpose['sum_purpose'] = data.groupby('purpose_type')['debt'].sum()

# добавляем столбец количества ячеек долга сгруппированных по целям кредита из основной таблицы data
debt_of_purpose['count_purpose'] = data.groupby('purpose_type')['debt'].count()

# вычесляем отношение количества к сумме и умножаем на 100
debt_of_purpose['result_purpose'] = debt_of_purpose['sum_purpose'] / debt_of_purpose['count_purpose'] * 100

# отображаем результат для анализа
debt_of_purpose.sort_values('result_purpose', ascending = False)['result_purpose']

purpose_type
автомобиль      9.359034
образование     9.220035
свадьба         8.003442
недвижимость    7.233373
Name: result_purpose, dtype: float64

Вот нам и понадобилось разбиение целей кредита по категориям. Как мы видим, среди 4 наших категорий, чаще всего платят в срок с целью кредита недвижимость, чаще просрочка у тех кому нужен кредит на авто и образование. Кредиты на свадьбу просрачивают реже - примерно как и в случае недвижимости.

**Вывод**

Отвечаем на гипотезу: `Как разные цели кредита влияют на его возврат в срок?`

Мы видим, что есть зависимость между целью кредита и его возвратом в срок.

Заемщики у которых цель кредита `автомобиль` и `образование` - чаще не возвращают кредит в срок по сравнению с заемщиками у которых цель кредита `свадьба`, `недвижимость` и `ремонт`. 

Заемщики у которых цель кредита `автомобиль` - самые рискованные. А заемщики с целью кредита `ремонт` - чаще других отдают кредит в срок.

## Шаг 4. Общий вывод

Да. Семейное положение и наличие детей в семье влияют на факт погашения кредита.
Не женатые заемщики и в гражданском браке имеют больший процент невозвратов кредитов в срок по сравнению с теми кто в разводе или овдовели.
Мы видим, что заемщики без детей чаще выплачивают кредит в срок, чем заемщики с детьми.

Лучшими заемщиками будут те кто овдовели и заемщики без детей.